# version for one pdf

In [1]:
import camelot
import pandas
pandas.options.display.max_colwidth = None

lattice mode is default, but doesn't recognise correctly transactions, only outer table, so I'll use it for getting header/footer eg start and end month balance

I'll use stream mode to get inner table based on coordinates where tables are expected in each page


In [2]:
# table_areas = top_left x, y, bottom_right x, y // 0,0 is bottom left
# read all pages in bottom 600x600 square
tables_transactions = camelot.read_pdf('private_test.pdf', flavor='stream', table_regions=['0,600,600,0'], pages='1-end')
tables_for_balance = camelot.read_pdf('private_test.pdf', pages='1-end')

getting to know the data

In [3]:
# tables[0].parsing_report

In [4]:
# camelot.plot(tables[0], kind='contour')

In [5]:
# camelot.plot(tables[0], kind='text')

In [6]:
# camelot.plot(tables[0], kind='grid')

# extract the balance at the beginning and end of the month

In [7]:
trans_begin = tables_for_balance[0].df # take 3 lines
begin_month_balance = ''.join(trans_begin.iat[1,0].split())


begin_month_balance[28:]

In [8]:
trans_end = tables_for_balance[-1].df # take 3 lines
end_month_balance = ''.join(trans_end.iat[-2,0].split())


end_month_balance[28:]

# get the list of all monthly transactions, clean the data

### convert each pdf page to dataframe and get the rows out of that and combine them all together


In [9]:
transactions_list = []
for table in tables_transactions:
    trans = table.df.values.tolist()
    for row in trans:
        transactions_list.append(row)
transactions = pandas.DataFrame(transactions_list)

### drop the columns with no data


In [10]:
transactions = transactions.drop(columns=[2,4,5])

### remove whitespace in the data

In [11]:
def remove_whitespace(x):
    if x is not None:
        x = x.split()
        x = ''.join(x)
    return x

transactions = transactions.applymap(remove_whitespace)

### rename columns

In [12]:
transactions.columns = ['info', 'date', 'out', 'in']

### remove non dates from date column, leave empty ones

In [13]:
transactions = transactions[transactions.date != 'Valuta']
transactions = transactions[transactions.date != 'zu']

### remove None values
they're of NoneType

In [14]:
transactions = transactions.fillna(value="")

### reset the index

In [15]:
transactions.index = range(len(transactions.index))

transactions[125:135]

# merge 3 rows of one transaction into one, discard the rest

### one row per transaction output example

transactions[transactions.date != ''][:2]

In [16]:
list_merged_transactions = []
use_this_row = 0 # we'll use 3 consecutive rows, and merge them 
current_info = ''
current_date = ''
current_in = ''
current_out = ''
new_row = []
for index, row in transactions.iterrows():
        if (row['date'] != ''):
            new_row = []
            use_this_row = 0
            current_info = row['info']
            current_date = row['date']
            current_in = row['in']
            current_out = row['out']
        else: 
            use_this_row += 1
            if use_this_row < 3: # unused row after row with date
                current_info += "\n" + row['info']
            if use_this_row == 3:
                list_merged_transactions.append([current_info, current_date, current_out, current_in])


In [17]:
transactions = pandas.DataFrame(list_merged_transactions)

In [ ]:
# transactions.to_csv("out.csv", sep='|')